In [1]:
import os
import sys
import numpy as np

In [2]:
def get_channel_dim(x):
    return x[-1]

def get_HW(x):
    return x[1]

用于计算pool层参数的个数，更新在params里，并且返回最后的feature map的形状

strides == 1时，采用1x1的卷积层然后对图像进行2x2的pooling，但是实际实现里面，并没有添加pooling，可能实际中根本不会令strides = 1

strides == 2时，用两个分支操作得到最后的feature map,这两个分支，分别代表奇数行列和偶数行列的信息。从参数个数的角度，两者是一样的

In [3]:
def factorized_reduction(inputs,filters,strides,params):
    assert filters % 2 == 0, ('Need even number of filters when using this factorized reduction')
    if strides == 1:
        params.append(inputs[-1]*1*1*filters)
        inputs = (inputs[0],inputs[1],filters)
        params.append(4*filters)
        return inputs
    
    #strides == 2
    params.append(inputs[-1]*1*1*filters/2)
    path1 = inputs[0]//2,inputs[1]//2,filters // 2
    
    params.append(inputs[-1]*1*1*filters/2)
    path2 = inputs[0]//2,inputs[1]//2,filters // 2  
    
    final_path = path1[0],path1[1],filters
    
    return final_path

定义一个NASCell类

其中的call函数返回这个cell输出的feature map的形状，并且加参数个数添加进params里

call这个函数，先使用maybe_calibrate_size函数对两个输入规范化，使得两个输入的H和W相同，并且通道数等于需要输出的channel数

随后，跟随整个cell处理tensor的流程，走一遍。计算其中的参数个数，并且记录下哪些层用作了其他层的input，存在used里面

最后将所有未在used里的中间输出通过channel合并，然后使用1x1的卷积，变成需要的channel数，即self._filter_size
始终还是不太明白，为何最后要添加channel数个，几乎所有我认为是1x1的卷积的地方都添加了这么一个数，到实现模型的时候，得注意看一下！！
{{{{{原来是添加一个batch——norm的操作}}}}}}

------------------------------------
maybe_calibrate_size是用来调整两个输入的size的，如果两个size的H和W不同，那么对H大的tensor进行一个factorized reduction，注意，这里经过reduction之后，输出的channel已经是out filters，否则，如果第一个的channel数还不是out filters，猜测进行一个1x1的卷积，但是不知为何最后要添加一个4*out_filters

对于第二个的channel数，如果还不一致，采取同样的操作

------------------------------------
_nas_cell 输入（输入的feature map的大小，当前层的id，输入层的id，操作的id，输出的channel数，参数统计）

值得注意的是，当前层的id和输入层的id起点并不统一，当前层id取0时，是第一个点，而输入层的id取0的，是上上个cell的输出
根据以上的信息，来更新总的参数个数，返回的应该是经过这个cell之后的feature map的形状，因为x和y形状一样，并且最后直接+起来，所以output直接就取x

由于在这种情况下，可能的操作数为5个，按照op_id从0到4，分别为 

3x3 separable conv
5x5 separable conv
3x3 avg pool
3x3 max pool
identity

在这个函数中，用out存储了每种的可能性，然后也就是说out里将会存5个元素，每个元素都是包含两个子元素的列表，第一个子元素表示输出的feature map的大小，第二个表示参数的个数


In [ ]:
class NASCell(object):
    def __init__(self, filters, dag, num_nodes, num_cells):
        self._filters = filters
        self._dag = dag
        self._num_nodes = num_nodes
        self._num_cells = num_cells
    
    def _maybe_calibrate_size(self,layers,out_filters,params):
        #make sure layers[0] and layers[1] have the same shapes.
        hw = [get_HW(layer) for layer in layers]
        c = [get_channel_dim(layer) for layer in layers]
        
        x = layers[0]
        if hw[0] != hw[1]:
            assert hw[0] == 2* hw[1]
            x = factorized_reduction(x,out_filters,2,params)
        elif c[0] != out_filters:
            params.append(1*1*c[0]*out_filters)
            x = x[0],x[1],out_filters
            params.append(4*out_filters)
        
        y = layers[1]
        
        if c[1]!= out_filters:
            params.append(1*1*c[0]*out_filters)
            y = y[0],y[1],out_filters
            params.append(4*out_filters)
        
        return [x,y]
    
    def _nas_sep_conv(self,x,curr_cell,prev_cell,filter_size,out_filters,stack_conv=2):
        params = 0
        for conv_id in range(stack_conv):
            inp_c = get_channel_dim(x)
            params += filter_size * filter_size * inp_c * 1 #depthwise
            params += inp_c * out_filters #pointwise
            params += out_filters * 4 #unknown????? why 4???? ?????????????????????????????????????
    
    def _nas_cell(self,x,curr_cell,prev_cell,op_id,out_filters,params):
        max_pool_c = get_channel_dim(x)
        max_pool_3 = x,0
        if max_pool_c != out_filters:
            max_pool_3 = (x,max_pool_c*out_filters + out_filters) #the mysterious 1x1 conv
        
        avg_pool_c = get_channel_dim(x)
        avg_pool_3 = x,0
        if avg_pool_c != out_filters:
            avg_pool_3 = (x,avg_pool_c*out_filters + out_filters)
        
        x_c = get_channel_dim(x)
        if x_c != out_filters:
            params.append(x_c*out_filters) #Here seems a problem, this should not be added since it has been added previously if op_id = 2 or 3
   ######################################################################################## 
   #if want to fix, remove the max_pool_c*out_filters and avg_pool_c*out_filters from max_pool_3 and avg_pool_3 respectively.
   #######################################################################################     
   ######################################################################################
        out = [
            self._nas_sep_conv(x,curr_cell,prev_cell,3,out_filters),
            self._nas_sep_conv(x,curr_cell,prev_cell,5,out_filters),
            avg_pool_3,
            max_pool_3,
            (x,0),
        ]
        out = out[op_id]
        params.append(out[1])
        return out[0]
    
    def __call__(self,inputs,filter_scaling=1,last_inputs=None,params=[]):
        
        self._filter_scaling = filter_scaling
        self._filter_size = int(self._filters * filter_scaling)
        num_nodes = self._num_nodes
        
        layers = [last_inputs, inputs]
        layers = self._maybe_calibrate_size(layers,self._filter_size,params)
        
        used = []
        for i in range(num_nodes):
            prev_layers = layers #maintain all inputs, add one element each iteration
            x_id = dag[4*i]
            x_op = dag[4*i+1]
            x = prev_layers[x_id]
            x = self._nas_cell(x,i,x_id,x_op,self._filter_size,params)
            
            if x_id not in used:
                used.append(x_id)
            
            y_id = dag[4*i+2]
            y_op = dag[4*i+3]
            y = prev_layers[y_id]
            y = self._nas_cell(y,i,y_id,y_op,self._filter_size,params)
            if y_id not in used:
                used.append(y_id)
            
            output = x
            layers.append(output)
        
        out = output[0],output[1],(num_nodes+2-len(used)) * self._filter_size
        
        params.append(1*1*out[-1]*self._filter_size)
        out = out[0],out[1],self._filter_size
        params.append(self._filter_size)
        
        return out

_build_aux_head函数

aux_net是一个feature map的形状 (H,W,C)

在这个函数里，对aux_net进行了改变，使得aux_net的形态变成了(1,1,10)，以便进行softmax的操作

第一步，似乎是进行了一个1x1的卷积，stride = 3，输出的channel数为128,不明白随后添加的128是什么参数

第二步，同样是1x1的卷积，stride = 1, 输出的channel数为768，同样最后添加了一个768，但是添加768之后似乎形状变成（1，1，768）

最后一步，从（1，1，768）变为（1，1，10），至于为何只需要10个参数，未知

In [1]:
def _build_aux_head(aux_net,params):
    aux_net = aux_net[0] // 3, aux_net[1] // 3, aux_net[2]
    params.append(1*1*aux_net[-1]*128)
    aux_net = aux_net[0],aux_net[1],128
    params.append(128)
    
    params.append(1*1*aux_net[-1]*768)
    aux_net = aux_net[0], aux_net[1],768
    params.append(768)
    
    aux_net = 1,1,768
    params.append(aux_net[0]*aux_net[1]*10)
    aux_net = (1,1,10)
    
    return aux_net
    

计算一个模型的参数个数

N = 2表示每个模块内normal cell copy 2次

num_cells = N* 3表示将这个模块copy 3次，其中并不包含需要加在中间的两次reduction cell的个数


total_num_cells = num_cells + 2，这里的+2究竟是加的reduction cell的个数还是加的最开始的两层，不得而知

reduction_layers里面保存了两个pool layer的index, reduction_layers和reduction cell是什么关系呢。

stem_multiplier is the channel multiplier for the preprocessing layer at the very beginning of the network.
最开始有一个增加channel数量的层，是3x3,stride=1。inputs表示输入的图片的规模，在这里是（32，32，3）。因此params第一个添加的是channel增加层的参数的个数. 第二个是做一个batch-norm一样的操作

In [ ]:
def calculate_model_params(inputs, conv_dag, reduc_dag, filters = 20, N=2,num_nodes=5,stem_multiplier=3):
    params = []
    num_cells = N*3 
    total_num_cells = num_cells + 2
    convolution_cell = NASCell(filters, conv_dag, num_nodes, total_num_cells)
    reduction_cell = NASCell(filters,reduc_dag,num_nodes,total_num_cells)
    
    reduction_layers = []
    for pool_num in range(1,3):
        layer_num = (float(pool_num) / (2+1)) * num_cells
        layer_num = int(layer_num)
        reduction_layers.append(layer_num)
    
    if len(reduction_layers) >= 2:
        aux_head_ceill_index = reduction_layers[1] # -1
    
    params.append(3*3*inputs[-1]*filters*stem_multiplier)
    params.append(filters*stem_multiplier)
    
    inputs = (inputs[0],inputs[1],filters*stem_multiplier)
    layers = [inputs,inputs]
    
    true_cell_num,filter_scaling = 0, 1
    
    for cell_num in range(num_cells):
        if cell_num in reduction_layers:
            filter_scaling *= 2
            inputs = factorized_reduction(inputs,filters*filter_scaling,2,params)
            layers = [layers[-1],inputs]
            inputs = reduction_cell(layers[-1],filter_scaling,layers[-2],params)
            layers = [layers[-1],inputs]
        inputs = convolution_cell(layers[-1],filter_scaling,layers[-2],params)
        layers = [layers[-1],inputs]
        if aux_head_ceill_index == cell_num:
            aux_logits = _build_aux_head(inputs,params) # turn the size of inputs to be (1,1,10) 
    
    params.append(inputs[-1]*10) #final softmax
    return sum(params)

给定一些架构，返回出一个列表，列表里每个元素都对应一个架构的参数的个数

In [ ]:
def calculate_params(arch_pool,filters=20,N=2,num_nodes=5,stem_multiplier=3):
    sizes = []
    
    for arch in arch_pool:
        conv_arch = arch[0]
        reduc_arch = arch[1]
        size = calculate_model_params( (32,32,3),conv_arch,reduc_arch,filters=filters,N=N,num_nodes = num_nodes,stem_multiplier=stem_multiplier)
        
        sizes.append(size)
    return sizes
        
        
        